In [7]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()


# 현재 파일의 위치를 기준으로 프로젝트 루트(backend) 경로를 계산하여 추가
root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

In [8]:
from datetime import datetime

from fastapi import APIRouter, Depends, HTTPException, status
from fastapi.responses import StreamingResponse
from sqlalchemy.orm import Session

from app.agents.multi_agent_graph import graph as multi_agent_graph
from app.deps import get_current_user, get_db
from app.models import Conversation, Message, User
from app.schemas import (
    ChatResponse,
    ConversationCreate,
    ConversationOut,
    MessageCreate,
    MessageOut,
)

router = APIRouter(prefix="/conversations", tags=["conversations"])

In [9]:
def _get_conversation(db: Session, user_id: int, conversation_id: int) -> Conversation:

    conversation = (  # -> 채팅방
        db.query(Conversation)
        .filter(Conversation.id == conversation_id, Conversation.user_id == user_id)
        .first()  # -> 대화는 어차피 1개지만 객체로 받기 위해 추가
    )
    if not conversation:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND, detail="채팅방을 찾을 수 없습니다."
        )
    return conversation

In [10]:
db_gen = get_db()
actual_db_session = next(db_gen)
try:
    # 3. 진짜 세션을 함수에 전달
    conversation = _get_conversation(db=actual_db_session, user_id=1, conversation_id=3)
    all_messages = conversation.messages
finally:
    # 4. 사용 후 닫기
    actual_db_session.close()

In [14]:
chat_history = []
        
for msg in all_messages:
        chat_history.append({"role": msg.role, "content": msg.content})

In [ ]:
chat_history

In [ ]:
def get_all_messages(conversation_id: int,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user)):

    conversation = _get_conversation(db, current_user.id, conversation_id)
    
    all_messages = conversation.messages
    return all_messages

In [11]:
@router.post("/{conversation_id}/messages")
async def create_message(
    conversation_id: int,
    payload: MessageCreate,
    db: Session = Depends(get_db),
    current_user: User = Depends(get_current_user),
):
    conversation = _get_conversation(db, current_user.id, conversation_id)

    user_message = Message(
        conversation_id=conversation.id, role="user", content=payload.content
    )
    db.add(user_message)

    if not conversation.title or conversation.title == "새 대화":
        conversation.title = payload.content.strip()[:40]

    conversation.updated_at = datetime.utcnow()
    db.flush()

    all_messages = conversation.messages
    chat_history = []

    for msg in all_messages:
        if msg.id != user_message.id:
            chat_history.append({"role": msg.role, "content": msg.content})

    async def event_generator():
        full_answer = ""
        try:
            async for event in multi_agent_graph.astream_events(
                {"query": payload.content, "chat_history": chat_history}, version="v2"
            ):
                if event["event"] == "on_chat_model_stream":
                    # router의 structured output 제외 (generate, llm 노드의 출력만 포함)
                    tags = event.get("tags", [])
                    name = event.get("name", "")

                    # structured output (router) 이벤트 건너뛰기
                    if "with_structured_output" in name or "structured" in name.lower():
                        continue

                    chunk = event["data"]["chunk"].content
                    if chunk:
                        full_answer += chunk
                        yield f"data: {json.dumps({'type': 'token', 'content': chunk})}\n\n"

            assistant_message = Message(
                conversation_id=conversation.id, role="assistant", content=full_answer
            )
            db.add(assistant_message)
            conversation.updated_at = datetime.utcnow()
            db.commit()
            db.refresh(conversation)
            db.refresh(user_message)
            db.refresh(assistant_message)

            # 완료 이벤트
            yield f"data: {json.dumps({'type': 'done', 'user_message_id': user_message.id, 'assistant_message_id': assistant_message.id, 'conversation_title': conversation.title})}\n\n"

        except Exception as e:
            db.rollback()
            yield f"data: {json.dumps({'type': 'error', 'message': str(e)})}\n\n"

    return StreamingResponse(event_generator(), media_type="text/event-stream")

In [17]:
from app.agents.multi_agent_graph import graph as multi_agent_graph


async def test_streaming():
    async for event in multi_agent_graph.astream_events(
        {"query": "안녕", "chat_history": []}, version="v2"
    ):
        print(event)
        print("==============")
        # if event["event"] == "on_chat_model_stream":
        #     name = event.get("name", "")
        #     chunk = event["data"]["chunk"].content
        #     print(f"[{name}] {repr(chunk)}")


await test_streaming()

{'event': 'on_chain_start', 'data': {'input': {'query': '안녕', 'chat_history': []}}, 'name': 'LangGraph', 'tags': [], 'run_id': '019bbd56-e6f7-7171-ac29-add987a4663b', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'query': '안녕', 'chat_history': []}}, 'name': '__start__', 'tags': ['graph:step:0', 'langsmith:hidden'], 'run_id': '019bbd56-e6fc-78a3-af9b-b4327f8a0169', 'metadata': {'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ('__start__',), 'langgraph_path': ('__pregel_pull', '__start__'), 'langgraph_checkpoint_ns': '__start__:2b491ae1-e04d-4672-20e6-f781f7ead801'}, 'parent_ids': ['019bbd56-e6f7-7171-ac29-add987a4663b']}
{'event': 'on_chain_start', 'data': {'input': {'query': '안녕', 'chat_history': []}}, 'name': 'router', 'tags': ['seq:step:2', 'langsmith:hidden'], 'run_id': '019bbd56-e6fd-7f93-8f5c-0bb413f49aa7', 'metadata': {'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ('__start__',), 'langgraph_p

In [12]:
# Depends(get_db) is only for FastAPI Routes. When testing manually, create the session directly.

db_gen = get_db()
db = next(db_gen)

try:
    # Assuming user_id=1 and conversation_id=3 exist for testing
    test_user_id = 1
    test_convo_id = 3

    conversation = _get_conversation(db=db, user_id=test_user_id, conversation_id=test_convo_id)
    print(f"Conversation found: ID {conversation.id}, Title: {conversation.title}")
except Exception as e:
    print(f"Error: {e}")
finally:
    db.close()

Conversation found: ID 3, Title: 5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있
